# Find ZP offset between NIRCam SCAs using the M92 data

Note: this version works with the M92 catalogs dubbed "redux2", i.e. catalogs derived from a second reduction of the original data, using an updated version of the pipeline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KernelDensity
%matplotlib notebook

In [ ]:
def m92_chip_number(data):
    chipnum = np.empty(len(data['x']), dtype='U2')
    # LONG LINE ACROSS CENTER
    x=np.arange(0,10000,1)
    m1 = (5722-4126)/(479-9849)
    b1 = 5722 - m1*(479)
    y1=m1*x+b1
    # FIRST
    x2=np.arange(1900,3000)
    m2 = (7445-3445)/(2821-2114)
    b2 = 7445 - 2821*m2
    y2=m2*x2+b2
    # SECOND
    x3 = np.arange(4500,6000)
    m3 = (7064-2976)/(5664 - 4806)
    b3 = 7064 - 5664*m3
    y3=m3*x3+b3
    # THIRD
    x4=np.arange(7500,8300)
    m4 = (6561-2552)/(8222-7498)
    b4 = 6561 - 8222*m4
    y4=m4*x4+b4
    yc1 = m1*data['x'] + b1
    yc2 = m2*data['x'] + b2
    yc3 = m3*data['x'] + b3
    yc4 = m4*data['x'] + b4
    # CHIP B2
    mb2 = (data['y'] > yc1) & (data['y'] > yc2)
    chipnum[mb2] = 'b2'
    # CHIP B1
    mb1 = (data['y'] < yc1) & (data['y'] > yc2)
    chipnum[mb1] = 'b1'
    # CHIP B4
    mb4 = (data['y'] > yc1) & (data['y'] < yc2) & (data['y'] > yc3)
    chipnum[mb4] = 'b4'
    # CHIP B3
    mb3 = (data['y'] < yc1) & (data['y'] < yc2) & (data['y'] > yc3)
    chipnum[mb3] = 'b3'
    # CHIP A3
    ma3 = (data['y'] > yc1)    & (data['y'] > yc4)& (data['y'] < yc3)
    chipnum[ma3] = 'a3'
    # CHIP A4
    ma4 = (data['y'] < yc1)    & (data['y'] > yc4)& (data['y'] < yc3)
    chipnum[ma4] = 'a4'
    # CHIP A1
    ma1 = (data['y'] > yc1)    & (data['y']< yc4)
    chipnum[ma1] = 'a1'
    # CHIP A2
    ma2 = (data['y'] < yc1)    & (data['y']< yc4)
    chipnum[ma2] = 'a2'
    return chipnum

## Read the M92 catalog

In [ ]:
df = pd.read_csv('/Users/gennaro/ERS_sandbox/Data/Full_cats/M92_redux2_all_columns.csv')

In [ ]:
df.columns

## Assign each star to its chip

In [ ]:
data = {'x':df['F150W_xbar'],'y':df['F150W_ybar']}
c = m92_chip_number(data)

In [ ]:
chips = [sca+num for sca in ['a','b'] for num in ['1','2','3','4']]

In [ ]:
f,ax = plt.subplots(1,1)
for chip in chips:
    BM = c == chip
    ax.scatter(df.loc[BM,'F150W_xbar'],df.loc[BM,'F150W_ybar'],s=1)

ax.set_aspect('equal')

### Apply some strict cuts in order to use only the very good stars

In [ ]:
f,ax = plt.subplots(1,1)

ax.scatter(df['F090W_mbar'],df['F090W_qbar'],s=1)

In [ ]:
BM_good = (df['F090W_mbar'] < -0.1) & (df['F150W_mbar'] < -0.1) & (df['F150W_qbar'] <.25) & (df['F090W_qbar'] <.25) \
          & (df['F150W_msig'] <0.1) & (df['F090W_msig'] <0.1) & (df['F090W_Ng'] > 2) & (df['F150W_Ng'] > 2)


#BM_good = BM_good | True

### Some diagnostic plots

In [ ]:
f,ax = plt.subplots(2,4,figsize=(10,6),sharex=True,sharey=True)

for axx,chip in zip(ax.reshape(-1),chips):
    BM = c == chip
    axx.scatter(df.loc[BM,'F090W_mbar']-df.loc[BM,'F150W_mbar'],df.loc[BM,'F150W_mbar'],s=1)
    axx.scatter(df.loc[BM&BM_good,'F090W_mbar']-df.loc[BM&BM_good,'F150W_mbar'],df.loc[BM&BM_good,'F150W_mbar'],s=1)
    axx.set_title(chip)

axx.set_xlim(-0.6,.7)
axx.set_ylim(-5,-17)
f.tight_layout()

In [ ]:
f,ax = plt.subplots(2,2,figsize=(10,5),sharex='col',sharey='col')

for chip in chips:
    BM = c == chip
    ax[0,0].hist(df.loc[BM,'F150W_mbar'],label=chip, bins = 100, histtype='step')
    ax[0,1].hist(df.loc[BM,'F090W_mbar'],label=chip, bins = 100, histtype='step')
    ax[1,0].hist(df.loc[BM&BM_good,'F150W_mbar'],label=chip, bins = 100, histtype='step')
    ax[1,1].hist(df.loc[BM&BM_good,'F090W_mbar'],label=chip, bins = 100, histtype='step')

ax[0,0].legend();
ax[0,1].legend();
ax[0,0].set_title('F150W, all')
ax[0,1].set_title('F090W, all')
ax[1,0].set_title('F150W, selected')
ax[1,1].set_title('F090W, selected')

ax[0,0].set_yscale('log')
ax[0,1].set_yscale('log')

f.tight_layout;

## Use kernel density estimate to find the peak of the luminosity function

### Try obtaining the optimal BW

In [ ]:
optimize = False

if optimize:
    bf = []
    for f in ['F150W_mbar','F090W_mbar']:
        bff = []
        for chip in chips:
            BM = c == chip
            x  = df.loc[BM&BM_good,f]

            BM2 = (x > -11) & (x < -8)
            x = x[BM2]
            print('Doing chip, filter:',chip,f)
            bandwidths = np.linspace(0.03, 0.13, 21)
            grid = GridSearchCV(KernelDensity(kernel='gaussian'),
                            {'bandwidth': bandwidths})
            grid.fit(np.array(x)[:, None]);
            print(grid.best_params_)
            bff.append(grid.best_params_)
        bf.append(bff)
    
    

### Use either the optimal or a ad-hoc bandwidth to create the KDE object

In [ ]:
kdes = []       
for i,f in enumerate(['F150W_mbar','F090W_mbar']):
    kkdes = []
    for j,chip in enumerate(chips):
        BM = c == chip
        x  = df.loc[BM&BM_good,f]

        BM2 = (x > -12) & (x < -8)
        x = x[BM2]
        print('Doing chip, filter:',chip,f)
        if optimize:
            print(bf[i][j])
            kde = KernelDensity(kernel='gaussian', bandwidth=bf[i][j]['bandwidth']).fit(x.values.reshape(-1, 1))
        else:
            kde = KernelDensity(kernel='gaussian', bandwidth=0.15).fit(x.values.reshape(-1, 1))
        kkdes.append(kde)
    kdes.append(kkdes)      

### Compute KDE values on a grid

In [ ]:
score_points = np.linspace(-12.5,-8,451).reshape(-1, 1)

scored = []
for kde in kdes:
    s = []
    for k in kde:
        print(k)
        s.append(k.score_samples(score_points))
    scored.append(s)


### Some plots

In [ ]:
f,ax = plt.subplots(1,2,figsize=(10,5))

for i,chip in enumerate(chips):
    BM = c == chip
    ax[0].plot(score_points,np.exp(scored[0][i]),label=chip)
    ax[1].plot(score_points,np.exp(scored[1][i]),label=chip)

ax[0].legend();
ax[1].legend();
ax[0].set_title('F150W')
ax[1].set_title('F090W')

### Find the magnitude corresponding to the peak

In [ ]:
max_mags = []
for ss in scored:
    mm = []
    for s in ss:
        imax = np.argmax(s)
        mag = score_points[imax,0]
        mm.append(mag)
    max_mags.append(mm)
        

In [ ]:
for i,f in enumerate(['F150W_mbar','F090W_mbar']):
    for j,chip in enumerate(chips):
        print('{} {} {:6.2f}'.format(f,chip,max_mags[i][j]))


## Test that the new relative ZP work by looking at the difference in color between teh B1 chip CMD ridge line vs. the other chips

In [ ]:
def median_RL(mag,col,start,end,binsize,minnum=10):
    
    medcol = []
    medmag = []
    while start < end:
        medmag.append(start+binsize/2.)
        BM = (mag<(start+binsize)) & (mag>start)
        if np.sum(BM) > minnum:
            medcol.append(np.median(col[BM]))
        else:
            medcol.append(np.nan)
        start = start+binsize
    
    return np.array(medcol),np.array(medmag)

In [ ]:
medcols = []
medmags = []
for chip in chips:
    BM = (c == chip) & BM_good

    mag = df.loc[BM,'F150W_mbar']
    col = df.loc[BM,'F090W_mbar']-df.loc[BM,'F150W_mbar']
    
    mc,mm = median_RL(mag,col,-15,-3,0.05,minnum=10)
    medcols.append(mc)
    medmags.append(mm)

medcols_corr = []
medmags_corr = []
for i,chip in enumerate(chips):
    BM = (c == chip) & BM_good

    delta_mag = max_mags[0][4] -max_mags[0][i]
    delta_col = max_mags[1][4] -max_mags[1][i] - (max_mags[0][4] -max_mags[0][i])
    
    mag = df.loc[BM,'F150W_mbar'] + delta_mag
    col = df.loc[BM,'F090W_mbar'] - df.loc[BM,'F150W_mbar'] + delta_col
    
    mc,mm = median_RL(mag,col,-15,-3,0.05,minnum=10)
    medcols_corr.append(mc)
    medmags_corr.append(mm)


In [ ]:
max_mags[0]

In [ ]:
f,ax = plt.subplots(2,4,figsize=(10,6),sharex=True,sharey=True)

for axx,chip,mc,mm in zip(ax.reshape(-1),chips,medcols,medmags):
    BM = c == chip
    axx.scatter(df.loc[BM,'F090W_mbar']-df.loc[BM,'F150W_mbar'],df.loc[BM,'F150W_mbar'],s=1)
    axx.scatter(df.loc[BM&BM_good,'F090W_mbar']-df.loc[BM&BM_good,'F150W_mbar'],df.loc[BM&BM_good,'F150W_mbar'],s=1)
    axx.plot(mc,mm,'-o',c='r',markersize=3)
    axx.set_title(chip)

axx.set_xlim(-0.4,.8)
axx.set_ylim(-5,-17)

f.text(0.5,0.04, "F090W-F150W (instr)", ha="center", va="center")
f.text(0.05,0.5, "F150W (instr)", ha="center", va="center", rotation=90)

f.tight_layout(rect=(0.05,0.05,1,1))

In [ ]:
f,ax = plt.subplots(1,2,figsize=(10,4),sharex=True,sharey=True)

for chip,mc,mcc in zip(chips,medcols,medcols_corr):
    ax[0].plot(medmags[0],mc-medcols[4],'-o',markersize=3,label=chip)
    ax[1].plot(medmags_corr[0],mcc-medcols_corr[4],'-o',markersize=3,label=chip)
    
ax[0].legend()
ax[1].legend()

ax[0].set_ylabel('Ridge Line color delta (chip - b1)')
ax[1].set_xlabel('F150W (instr)')
ax[0].set_xlabel('F150W (instr)')

ax[0].set_title('Before corr')
ax[1].set_title('After corr');

## Check the looks of the corrected vs uncorrected CMD

In [ ]:
for i,chip in enumerate(chips):
    BM = (c == chip) 

    
    df.loc[BM,'F150W_mbar_new'] = df.loc[BM,'F150W_mbar'] + max_mags[0][4] -max_mags[0][i]
    df.loc[BM,'F090W_mbar_new'] = df.loc[BM,'F090W_mbar'] + max_mags[1][4] -max_mags[1][i]
    
    


In [ ]:
f,ax = plt.subplots(1,2,figsize=(8,8),sharex=True,sharey=True)

ax[0].scatter(df['F090W_mbar']-df['F150W_mbar'],df['F150W_mbar'],s=1)
ax[1].scatter(df['F090W_mbar_new']-df['F150W_mbar_new'],df['F150W_mbar_new'],s=1)

ax[0].set_xlim(-0.5,1)
ax[0].set_ylim(-7,-17)

ax[0].set_title('Before corr')
ax[1].set_title('After corr')

f.text(0.5,0.04, "F090W-F150W (instr)", ha="center", va="center")
f.text(0.05,0.5, "F150W (instr)", ha="center", va="center", rotation=90)

f.tight_layout(rect=(0.05,0.05,.9,1))